In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
# import datasets
# from datasets import load_metric
from sklearn.metrics import accuracy_score
import torch

# 1. 读取文件

In [3]:
file_name='情感分析3.0.txt'

data = []
with open(file_name, 'r', encoding='gb18030') as f:
    lines = f.readlines()
    for i in range(0, len(lines), 2):
        text = lines[i].strip()
        sentiment = int(lines[i + 1].strip())
        if sentiment == -1:
            sentiment = 0
        data.append({'text': text, 'sentiment': sentiment})
df = pd.DataFrame(data)
# 将数据集拆分为训练集和验证集
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 2.准备库 划分训练集和验证集


In [6]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True)


In [7]:


class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_df['sentiment'].tolist())
val_dataset = SentimentDataset(val_encodings, val_df['sentiment'].tolist())


# 3. 初始参数微调 

In [14]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model_ori = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# 设置训练参数
training_args = TrainingArguments(
    output_dir='./results',                 # 输出目录
    num_train_epochs=10,                     # 训练的总轮数
    per_device_train_batch_size=32,          # 每个设备的训练批次大小
    per_device_eval_batch_size=32,           # 每个设备的评估批次大小
    warmup_steps=500,                       # 预热步骤
    weight_decay=0.01,                      # 权重衰减
    logging_dir='./logs',                   # 日志目录
    logging_steps=10,                       # 日志记录步骤
    evaluation_strategy='steps',            # 评估策略
    eval_steps=100,                         # 评估步骤
    load_best_model_at_end=True,            # 训练结束时加载最佳模型
)

# 创建训练器
trainer = Trainer(
    model=model_ori,                           # 要微调的模型
    args=training_args,                    # 训练参数
    train_dataset=train_dataset,           # 训练数据集
    eval_dataset=val_dataset,              # 评估数据集
)

# 微调模型
trainer.train()
eval_result = trainer.evaluate()
print("Evaluation results:", eval_result)

# # 保存模型和分词器
# model.save_pretrained("saved_model_directory")
# tokenizer.save_pretrained("saved_model_directory")


loading configuration file config.json from cache at C:\Users\1/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\1/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClas

Step,Training Loss,Validation Loss
100,0.414100,0.346926
200,0.305400,0.331076
300,0.257700,0.327422
400,0.284100,0.338233
500,0.218300,0.331523
600,0.152100,0.366274
700,0.091200,0.431159
800,0.063100,0.523039


***** Running Evaluation *****
  Num examples = 712
  Batch size = 32
***** Running Evaluation *****
  Num examples = 712
  Batch size = 32
***** Running Evaluation *****
  Num examples = 712
  Batch size = 32
***** Running Evaluation *****
  Num examples = 712
  Batch size = 32
***** Running Evaluation *****
  Num examples = 712
  Batch size = 32
Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 712
  Batch size = 32
***** Running Evaluation *****
  Num examples = 712
  Batch size = 32
***** Running Evaluation *****
  Num examples = 712
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-500 (score: 0.33152300119400024).
***** Running Evaluation *****
  Num examples = 712
  Batch size = 32


Evaluation results: {'eval_loss': 0.33152300119400024, 'eval_runtime': 4.529, 'eval_samples_per_second': 157.208, 'eval_steps_per_second': 5.078, 'epoch': 10.0}


In [37]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_df['sentiment'].tolist())
val_dataset = SentimentDataset(val_encodings, val_df['sentiment'].tolist())


In [108]:
# import time

# output_dir = f'./results-{int(time.time())}'

# training_args = TrainingArguments(
#     output_dir=output_dir,                 # 使用包含时间戳的唯一输出目录
#     num_train_epochs=5,                    # 增加训练轮数
#     per_device_train_batch_size=16,        # 更改训练批次大小
#     per_device_eval_batch_size=16,         # 更改评估批次大小
#     warmup_steps=800,                      # 增加预热步骤
#     weight_decay=0.02,                     # 调整权重衰减
#     logging_dir='./logs',                  # 日志目录
#     logging_steps=10,                      # 日志记录步骤
#     evaluation_strategy='steps',           # 评估策略
#     eval_steps=100,                        # 评估步骤
#     load_best_model_at_end=True,           # 训练结束时加载最佳模型
# )

# # 初始化模型和分词器
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# # 使用新的参数训练模型
# trainer_1 = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,              # 评估数据集
# )

# # 微调模型
# trainer_1.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at C:\Users\1/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weigh

Step,Training Loss,Validation Loss
100,0.326100,0.350732
200,0.405300,0.329657
300,0.341200,0.320315
400,0.337900,0.306652
500,0.248200,0.322431
600,0.225900,0.370387
700,0.233800,0.349566
800,0.233800,0.427196


***** Running Evaluation *****
  Num examples = 712
  Batch size = 16
***** Running Evaluation *****
  Num examples = 712
  Batch size = 16
***** Running Evaluation *****
  Num examples = 712
  Batch size = 16
***** Running Evaluation *****
  Num examples = 712
  Batch size = 16
***** Running Evaluation *****
  Num examples = 712
  Batch size = 16
Saving model checkpoint to ./results-1681870503\checkpoint-500
Configuration saved in ./results-1681870503\checkpoint-500\config.json
Model weights saved in ./results-1681870503\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 712
  Batch size = 16
***** Running Evaluation *****
  Num examples = 712
  Batch size = 16
***** Running Evaluation *****
  Num examples = 712
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results-1681870503\checkpoint-500 (score: 0.32243070006370544).


TrainOutput(global_step=890, training_loss=0.3180589532584287, metrics={'train_runtime': 472.2001, 'train_samples_per_second': 30.157, 'train_steps_per_second': 1.885, 'total_flos': 1455278562422400.0, 'train_loss': 0.3180589532584287, 'epoch': 5.0})

# 测试模型

In [29]:
new_texts = ['人福医药：布洛芬作为常用解热镇痛药，是公司众多医药产品之一','汉口银行延迟披露2019年报 未经审计的不良贷款率达1.75%']

def predict_sentiment(texts, model, tokenizer):
    # 确保模型在正确的设备上
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    inputs = inputs.to(device)  # 将输入数据移动到相同的设备

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        
    return preds.tolist()

sentiment_predictions = predict_sentiment(new_texts, model_ori, tokenizer)
print("Sentiment predictions:", sentiment_predictions)


Sentiment predictions: [1, 0]


In [20]:
from torch.utils.data import DataLoader
def get_predictions(model, tokenizer, dataset, device="cuda"):
    model.to(device)
    model.eval()

    dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
    predictions = []

    with torch.no_grad():
        for batch in dataloader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).tolist()
            predictions.extend(preds)

    return predictions


In [22]:

import numpy as np
from datasets import load_metric
from sklearn.metrics import accuracy_score

metric = load_metric("accuracy")

C:\Users\1\AppData\Local\Temp\ipykernel_3280\3442899734.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


# 2.测试准确率

SyntaxError: unexpected EOF while parsing (2378483863.py, line 20)

In [23]:

train_preds = get_predictions(model_ori, tokenizer, train_dataset)
eval_preds = get_predictions(model_ori, tokenizer, val_dataset)

train_accuracy = metric.compute(predictions=train_preds, references=train_dataset[:]["labels"])
eval_accuracy = metric.compute(predictions=eval_preds, references=val_dataset[:]["labels"])
print("Training accuracy:", train_accuracy)
print("Evaluation accuracy:", eval_accuracy)


Training accuracy: {'accuracy': 0.9287219101123596}
Evaluation accuracy: {'accuracy': 0.8918539325842697}


### 3.保存模型

In [25]:
# 保存模型和分词器
model_ori.save_pretrained("saved_model_directory_2")
tokenizer.save_pretrained("saved_model_directory_2")

Configuration saved in saved_model_directory_2\config.json
Model weights saved in saved_model_directory_2\pytorch_model.bin
tokenizer config file saved in saved_model_directory_2\tokenizer_config.json
Special tokens file saved in saved_model_directory_2\special_tokens_map.json


('saved_model_directory_2\\tokenizer_config.json',
 'saved_model_directory_2\\special_tokens_map.json',
 'saved_model_directory_2\\vocab.txt',
 'saved_model_directory_2\\added_tokens.json',
 'saved_model_directory_2\\tokenizer.json')

#### 4.加载模型

In [30]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

# 加载模型和分词器
loaded_model = DistilBertForSequenceClassification.from_pretrained("saved_model_directory_2")
loaded_tokenizer = DistilBertTokenizerFast.from_pretrained("saved_model_directory_2")


loading configuration file saved_model_directory_2\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file saved_model_directory_2\pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at saved_model_directory_2.
If your

In [31]:
sentiment_predictions = predict_sentiment(new_texts, loaded_model, loaded_tokenizer)
print("Sentiment predictions:", sentiment_predictions)

Sentiment predictions: [1, 0]
